In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime

OUTPUT_FILE = r"E:\Daily market data\Market_Data_Combined.xlsx"

# --------------------------
# NIFTY-50 SYMBOLS
# --------------------------
nifty50_symbols = {
    "RELIANCE.NS": "Reliance",
    "TCS.NS": "TCS",
    "HDFCBANK.NS": "HDFC Bank",
    "ICICIBANK.NS": "ICICI Bank",
    "INFY.NS": "Infosys",
    "HINDUNILVR.NS": "HUL",
    "SBIN.NS": "SBI",
    "ITC.NS": "ITC",
    "BHARTIARTL.NS": "Bharti Airtel",
    "KOTAKBANK.NS": "Kotak Bank",
    "AXISBANK.NS": "Axis Bank",
    "LT.NS": "L&T",
    "BAJFINANCE.NS": "Bajaj Finance",
    "BAJAJFINSV.NS": "Bajaj Finserv",
    "SUNPHARMA.NS": "Sun Pharma",
    "TITAN.NS": "Titan",
    "MARUTI.NS": "Maruti",
    "M&M.NS": "M&M",
    "ULTRACEMCO.NS": "UltraTech",
    "NESTLEIND.NS": "Nestle",
    "ASIANPAINT.NS": "Asian Paints",
    "TATAMOTORS.NS": "Tata Motors",
    "TATASTEEL.NS": "Tata Steel",
    "WIPRO.NS": "Wipro",
    "HCLTECH.NS": "HCL Tech",
    "TECHM.NS": "Tech Mahindra",
    "POWERGRID.NS": "Power Grid",
    "COALINDIA.NS": "Coal India",
    "ONGC.NS": "ONGC",
    "JSWSTEEL.NS": "JSW Steel",
    "DIVISLAB.NS": "Divis Labs",
    "DRREDDY.NS": "Dr Reddy",
    "HEROMOTOCO.NS": "Hero MotoCorp",
    "EICHERMOT.NS": "Eicher Motors",
    "GRASIM.NS": "Grasim",
    "HDFCLIFE.NS": "HDFC Life",
    "SBILIFE.NS": "SBI Life",
    "BRITANNIA.NS": "Britannia",
    "ADANIPORTS.NS": "Adani Ports",
    "APOLLOHOSP.NS": "Apollo Hospitals",
    "CIPLA.NS": "Cipla",
    "BAJAJ-AUTO.NS": "Bajaj Auto",
    "SHREECEM.NS": "Shree Cement",
    "INDUSINDBK.NS": "IndusInd Bank",
    "BPCL.NS": "BPCL",
    "UPL.NS": "UPL",
    "TATACONSUM.NS": "Tata Consumer"
}

INDEX_SYMBOL = "^NSEI"
EURINR = "EURINR=X"

all_symbols = list(nifty50_symbols.keys()) + [INDEX_SYMBOL, EURINR]

# --------------------------
# FUNCTION: INITIAL DOWNLOAD
# --------------------------
def download_10yr_data():
    print("Downloading full 10-year history...")
    data = yf.download(all_symbols, period="10y")["Close"]

    rename_map = {s: n for s, n in nifty50_symbols.items()}
    rename_map[INDEX_SYMBOL] = "NIFTY Index"
    rename_map[EURINR] = "EURINR"

    data = data.rename(columns=rename_map)

    # Save ALL_DATA
    data.to_excel(OUTPUT_FILE, sheet_name="ALL_DATA")

    # Create pivot
    create_pivot(data)
    print("Initial Excel created.")


# --------------------------
# FUNCTION: DAILY UPDATE
# --------------------------
def daily_update():
    print("Running daily update...")
    today = datetime.now().strftime("%Y-%m-%d")
    latest = yf.download(all_symbols, period="1d")["Close"]

    rename_map = {s: n for s, n in nifty50_symbols.items()}
    rename_map[INDEX_SYMBOL] = "NIFTY Index"
    rename_map[EURINR] = "EURINR"
    latest = latest.rename(columns=rename_map)

    old = pd.read_excel(OUTPUT_FILE, sheet_name="ALL_DATA", index_col=0)

    if today not in old.index.astype(str):
        final = pd.concat([old, latest])
    else:
        final = old

    with pd.ExcelWriter(OUTPUT_FILE, engine="openpyxl", mode="w") as writer:
        final.to_excel(writer, sheet_name="ALL_DATA")
        create_pivot(final)

    print("Daily update completed.")


# --------------------------
# FUNCTION: CREATE PIVOT
# --------------------------
def create_pivot(df):
    # Make sure DATE is index
    pivot = df.copy()
    pivot.index.name = "DATE"

    # Reset index for pivot layout
    pivot_reset = pivot.reset_index()

    # Save pivot sheet
    with pd.ExcelWriter(OUTPUT_FILE, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
        pivot_reset.to_excel(writer, sheet_name="PIVOT", index=False)


# --------------------------
# MAIN EXECUTION
# --------------------------
try:
    pd.read_excel(OUTPUT_FILE)
    daily_update()
except FileNotFoundError:
    download_10yr_data()



C:\Users\Swarupa\AppData\Local\Temp\5\ipykernel_10876\1029677550.py:70: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(all_symbols, period="10y")["Close"]
[*********************100%***********************]  49 of 49 completed


Initial Excel created.


In [2]:
import sys
print(sys.executable)


C:\Users\Swarupa\anaconda3\envs\ttp_env1\python.exe
